In [2]:
import pandas as pd
import json

In [12]:
path = r"C:\Users\pc\OneDrive\Desktop\movie-Box-hit-predictor\data_cleaned\first_cleaned_dataset.csv"
clean_df1 = pd.read_csv(path)
clean_df1.columns

Index(['Unnamed: 0', 'budget', 'popularity', 'vote_count',
       'top_cast_avg_rating', 'director_encoded', 'release_year',
       'release_month', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western', 'title', 'genres', 'director', 'top_cast',
       'release_date', 'vote_average', 'revenue', 'budget_log', 'revenue_log',
       'profit_ratio'],
      dtype='object')

In [11]:
path_2 = r"C:\Users\pc\OneDrive\Desktop\movie-Box-hit-predictor\data_cleaned\second_cleaned_dataset.csv"
clean_df2 = pd.read_csv(path_2)
clean_df2.columns

Index(['Unnamed: 0', 'title', 'budget_log', 'revenue_log', 'profit_ratio',
       'popularity', 'vote_average', 'vote_count', 'director_encoded',
       'top_cast_avg_rating', 'release_year', 'release_month', 'Action',
       'Action.1', 'Action.2', 'Action.3', 'Adventure', 'Adventure.1',
       'Adventure.2', 'Adventure.3', 'Animation', 'Animation.1', 'Animation.2',
       'Animation.3', 'Comedy', 'Comedy.1', 'Comedy.2', 'Comedy.3', 'Crime',
       'Crime.1', 'Crime.2', 'Crime.3', 'Documentary', 'Documentary.1',
       'Documentary.2', 'Documentary.3', 'Drama', 'Drama.1', 'Drama.2',
       'Drama.3', 'Family', 'Family.1', 'Family.2', 'Family.3', 'Fantasy',
       'Fantasy.1', 'Fantasy.2', 'Fantasy.3', 'Foreign', 'Foreign.1',
       'Foreign.2', 'Foreign.3', 'History', 'History.1', 'History.2',
       'History.3', 'Horror', 'Horror.1', 'Horror.2', 'Horror.3', 'Music',
       'Music.1', 'Music.2', 'Music.3', 'Mystery', 'Mystery.1', 'Mystery.2',
       'Mystery.3', 'Romance', 'Romance.1'

In [13]:
# Compare columns between dataset 1 and 2
set1 = set(clean_df1.columns)
set2 = set(clean_df2.columns)

common_cols = set1.intersection(set2)
only_in_df1 = set1 - set2
only_in_df2 = set2 - set1

print("✅ COMMON COLUMNS (in both datasets):")
print(sorted(common_cols))
print("\n➕ ONLY IN DATASET 1:")
print(sorted(only_in_df1))
print("\n➕ ONLY IN DATASET 2:")
print(sorted(only_in_df2))


✅ COMMON COLUMNS (in both datasets):
['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'Unnamed: 0', 'War', 'Western', 'budget_log', 'director_encoded', 'popularity', 'profit_ratio', 'release_month', 'release_year', 'revenue_log', 'title', 'top_cast_avg_rating', 'vote_average', 'vote_count']

➕ ONLY IN DATASET 1:
['budget', 'director', 'genres', 'release_date', 'revenue', 'top_cast']

➕ ONLY IN DATASET 2:
['Action.1', 'Action.2', 'Action.3', 'Adventure.1', 'Adventure.2', 'Adventure.3', 'Animation.1', 'Animation.2', 'Animation.3', 'Comedy.1', 'Comedy.2', 'Comedy.3', 'Crime.1', 'Crime.2', 'Crime.3', 'Documentary.1', 'Documentary.2', 'Documentary.3', 'Drama.1', 'Drama.2', 'Drama.3', 'Family.1', 'Family.2', 'Family.3', 'Fantasy.1', 'Fantasy.2', 'Fantasy.3', 'Foreign.1', 'Foreign.2', 'Foreign.3', 'History.1', 'History.2', 'History.3', '

In [14]:
import numpy as np
import pandas as pd

# ✅ List of all main genre names
genre_cols = [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
    'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
    'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
]

# ✅ Main features
feature_cols = [
    'title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count',
    'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month',
    'budget_log', 'revenue_log', 'profit_ratio'
]

# 🩵 Step 1: Fix genre columns in clean_df2
for g in genre_cols:
    dupes = [c for c in clean_df2.columns if c.startswith(g)]
    if dupes:
        clean_df2[g] = clean_df2[dupes].sum(axis=1).clip(0, 1)
        clean_df2.drop(columns=dupes, inplace=True, errors='ignore')
    else:
        clean_df2[g] = 0  # add if not found

# 🩵 Step 2: Add missing numeric columns in both
for df in [clean_df1, clean_df2]:
    for col in ['budget', 'revenue']:
        if col not in df.columns:
            df[col] = np.expm1(df[col + '_log']) if col + '_log' in df.columns else 0

    if 'budget_log' not in df.columns:
        df['budget_log'] = np.log1p(df['budget'])
    if 'revenue_log' not in df.columns:
        df['revenue_log'] = np.log1p(df['revenue'])
    if 'profit_ratio' not in df.columns:
        df['profit_ratio'] = (df['revenue'] - df['budget']) / df['budget'].replace(0, np.nan)

# 🩵 Step 3: Add any missing genre columns
for g in genre_cols:
    if g not in clean_df1.columns:
        clean_df1[g] = 0
    if g not in clean_df2.columns:
        clean_df2[g] = 0

# 🩵 Step 4: Align columns
keep_cols = feature_cols + genre_cols
clean_df1 = clean_df1.reindex(columns=keep_cols, fill_value=0)
clean_df2 = clean_df2.reindex(columns=keep_cols, fill_value=0)

# 🩵 Step 5: Combine datasets
combined_df = pd.concat([clean_df1, clean_df2], ignore_index=True)

print(f"✅ Combined dataset shape: {combined_df.shape}")
print(f"✅ Columns: {combined_df.columns.tolist()}")
combined_df.head()


✅ Combined dataset shape: (49525, 33)
✅ Columns: ['title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count', 'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month', 'budget_log', 'revenue_log', 'profit_ratio', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']


,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Pirates of the Caribbean: At World's End,300000000.0,961000000.0,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1,John Carter,260000000.0,284139100.0,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,0,1,0,0,0,0
2,Spider-Man 3,258000000.0,890871626.0,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,Tangled,260000000.0,591794936.0,48.681969,7.4,3330.0,257,6.636667,2010.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,Superman Returns,270000000.0,391081192.0,57.925623,5.4,1400.0,255,5.685556,2006.0,6.0,...,0,0,0,0,0,1,0,0,0,0


In [15]:
combined_df.columns

Index(['title', 'budget', 'revenue', 'popularity', 'vote_average',
       'vote_count', 'director_encoded', 'top_cast_avg_rating', 'release_year',
       'release_month', 'budget_log', 'revenue_log', 'profit_ratio', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')

In [16]:
combined_df.head(50)

,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Pirates of the Caribbean: At World's End,300000000.0,9.610000e+08,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1,John Carter,260000000.0,2.841391e+08,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,0,1,0,0,0,0
2,Spider-Man 3,258000000.0,8.908716e+08,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,Tangled,260000000.0,5.917949e+08,48.681969,7.4,3330.0,257,6.636667,2010.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,Superman Returns,270000000.0,3.910812e+08,57.925623,5.4,1400.0,255,5.685556,2006.0,6.0,...,0,0,0,0,0,1,0,0,0,0
5,Quantum of Solace,200000000.0,5.860907e+08,107.928811,6.1,2965.0,1284,6.271936,2008.0,10.0,...,0,0,0,0,0,0,0,1,0,0
6,Pirates of the Caribbean: Dead Man's Chest,200000000.0,1.065660e+09,145.847379,7.0,5246.0,714,6.271936,2006.0,6.0,...,0,0,0,0,0,0,0,0,0,0
7,The Lone Ranger,255000000.0,8.928991e+07,49.046956,5.9,2311.0,714,6.271936,2013.0,7.0,...,0,0,0,0,0,0,0,0,0,1
8,Man of Steel,225000000.0,6.628455e+08,99.398009,6.5,6359.0,2175,6.238889,2013.0,6.0,...,0,0,0,0,0,1,0,0,0,0
9,The Chronicles of Narnia: Prince Caspian,225000000.0,4.196514e+08,53.978602,6.3,1630.0,74,6.300000,2008.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Save as CSV (recommended)
combined_df.to_csv("combined_movies_cleaned.csv", index=False)

print("✅ Saved as 'combined_movies_cleaned.csv'")


✅ Saved as 'combined_movies_cleaned.csv'


In [17]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49525 entries, 0 to 49524
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                49525 non-null  object 
 1   budget               49525 non-null  float64
 2   revenue              49525 non-null  float64
 3   popularity           49525 non-null  float64
 4   vote_average         49525 non-null  float64
 5   vote_count           49525 non-null  float64
 6   director_encoded     49525 non-null  int64  
 7   top_cast_avg_rating  49525 non-null  float64
 8   release_year         49525 non-null  float64
 9   release_month        49525 non-null  float64
 10  budget_log           49525 non-null  float64
 11  revenue_log          49525 non-null  float64
 12  profit_ratio         48564 non-null  float64
 13  Action               49525 non-null  int64  
 14  Adventure            49525 non-null  int64  
 15  Animation            49525 non-null 

In [19]:
import pandas as pd

# Load your clean combined dataset
df = pd.read_csv(r'C:\Users\pc\OneDrive\Desktop\movie-Box-hit-predictor\data_cleaned\combined_movies_cleaned.csv')

# Check structure
print(df.shape)
df.info()

# Quick null + type summary
print("\n🧭 Missing values:")
print(df.isna().sum().sort_values(ascending=False).head(10))

print("\n🔢 Data types count:")
print(df.dtypes.value_counts())

# Sample
display(df.head(3))


(49525, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49525 entries, 0 to 49524
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                49525 non-null  object 
 1   budget               49525 non-null  float64
 2   revenue              49525 non-null  float64
 3   popularity           49525 non-null  float64
 4   vote_average         49525 non-null  float64
 5   vote_count           49525 non-null  float64
 6   director_encoded     49525 non-null  int64  
 7   top_cast_avg_rating  49525 non-null  float64
 8   release_year         49525 non-null  float64
 9   release_month        49525 non-null  float64
 10  budget_log           49525 non-null  float64
 11  revenue_log          49525 non-null  float64
 12  profit_ratio         48564 non-null  float64
 13  Action               49525 non-null  int64  
 14  Adventure            49525 non-null  int64  
 15  Animation            495

,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Pirates of the Caribbean: At World's End,300000000.0,961000000.0,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0
1,John Carter,260000000.0,284139100.0,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,0,1,0,0,0,0
2,Spider-Man 3,258000000.0,890871626.0,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Check for any non-numeric values in numeric columns
import numpy as np

for col in df.select_dtypes(include=[np.number]).columns:
    non_numeric = df[col].apply(lambda x: isinstance(x, str)).sum()
    if non_numeric > 0:
        print(f"⚠️ Column {col} has {non_numeric} string values")

print("✅ Numeric check complete!")


✅ Numeric check complete!


In [21]:
df.describe().T[['mean', 'std', 'min', 'max']].sort_values(by='max', ascending=False).head(10)


,mean,std,min,max
revenue,1.584016e+07,7.431029e+07,0.0,2.787965e+09
budget,5.962487e+06,2.061678e+07,0.0,3.800000e+08
profit_ratio,8.128681e+02,8.198133e+04,-1.0,1.239638e+07
vote_count,1.471375e+02,5.655438e+02,0.0,1.407500e+04
director_encoded,2.297488e+03,5.805508e+02,0.0,2.528000e+03
release_year,1.972643e+03,9.547553e+00,1925.0,2.017000e+03
popularity,4.210422e+00,1.058570e+01,0.0,8.755813e+02
revenue_log,3.351709e+00,6.721579e+00,0.0,2.174858e+01
budget_log,3.797218e+00,6.832808e+00,0.0,1.975568e+01
release_month,1.474548e+00,1.865669e+00,1.0,1.200000e+01


In [22]:
import numpy as np

# Fix negative profit ratios (e.g., -1 → 0)
df['profit_ratio'] = df['profit_ratio'].clip(lower=0)

# Add log-scaled profit ratio
df['profit_ratio_log'] = np.log1p(df['profit_ratio'])

# Replace any remaining NaNs
df = df.fillna({
    'profit_ratio': 0,
    'profit_ratio_log': 0
})

# Confirm everything looks clean
print(df[['profit_ratio', 'profit_ratio_log']].describe())



       profit_ratio  profit_ratio_log
count  4.952500e+04      49525.000000
mean   7.972018e+02          0.152552
std    8.118210e+04          0.561885
min    0.000000e+00          0.000000
25%    0.000000e+00          0.000000
50%    0.000000e+00          0.000000
75%    0.000000e+00          0.000000
max    1.239638e+07         16.332915


In [31]:
kaggle_df = df

In [1]:
print("hello")

hello


In [24]:
import pandas as pd
path = r"C:\Users\pc\OneDrive\Desktop\movie-Box-hit-predictor\data_cleaned\tmdb_api_movies_enriched.csv"
tmdb_api_data = pd.read_csv(path)
tmdb_api_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'id', 'budget', 'revenue',
       'popularity', 'vote_average', 'vote_count', 'release_date', 'genre_ids',
       'adult', 'original_language', 'release_year', 'release_month',
       'director_encoded', 'top_cast_avg_rating', 'budget_log', 'revenue_log',
       'profit_ratio'],
      dtype='object')

In [25]:
tmdb_df = tmdb_api_data

In [30]:
# === CLEAN AND ALIGN TMDB DATASET ===
import pandas as pd
import numpy as np

# 1️⃣ Keep only movies with nonzero budget and revenue
tmdb_filtered = tmdb_df[
    (tmdb_df['budget'] > 0) & 
    (tmdb_df['revenue'] > 0)
].copy()
print(f"✅ Filtered TMDb movies: {tmdb_filtered.shape}")

# 2️⃣ Create engineered features (logs + ratios)
tmdb_filtered['budget_log'] = np.log1p(tmdb_filtered['budget'])
tmdb_filtered['revenue_log'] = np.log1p(tmdb_filtered['revenue'])
tmdb_filtered['profit_ratio'] = (tmdb_filtered['revenue'] - tmdb_filtered['budget']) / tmdb_filtered['budget']
tmdb_filtered['profit_ratio_log'] = np.log1p(tmdb_filtered['profit_ratio'].clip(lower=0))

# 3️⃣ Fix data types
tmdb_filtered['adult'] = tmdb_filtered['adult'].astype(int)
tmdb_filtered['release_year'] = tmdb_filtered['release_year'].astype(int)
tmdb_filtered['release_month'] = tmdb_filtered['release_month'].astype(int)
# 4️⃣ Handle missing values and fix director column type safely

# If director_encoded is text (e.g., director names), encode it numerically
if tmdb_filtered['director_encoded'].dtype == 'object':
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    tmdb_filtered['director_encoded'] = (
        tmdb_filtered['director_encoded'].fillna('Unknown')
    )
    tmdb_filtered['director_encoded'] = le.fit_transform(tmdb_filtered['director_encoded'])

else:
    # If it's already numeric but has NaNs
    tmdb_filtered['director_encoded'] = tmdb_filtered['director_encoded'].fillna(0).astype(int)

# Fill missing top cast rating
tmdb_filtered['top_cast_avg_rating'] = tmdb_filtered['top_cast_avg_rating'].fillna(
    tmdb_filtered['top_cast_avg_rating'].mean()
)

# 5️⃣ Map TMDb genre_ids → genre name columns (like in Kaggle)
genre_map = {
    28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime',
    99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History',
    27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction',
    10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'
}

for gid, name in genre_map.items():
    tmdb_filtered[name] = tmdb_filtered['genre_ids'].apply(lambda x: 1 if str(gid) in str(x) else 0)

# 6️⃣ Keep only relevant columns (align with Kaggle dataset)
cols_to_keep = [
    'title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count',
    'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month',
    'budget_log', 'revenue_log', 'profit_ratio', 'profit_ratio_log'
] + list(genre_map.values())

tmdb_cleaned = tmdb_filtered[cols_to_keep].copy()
print(f"✅ Cleaned TMDb dataset shape: {tmdb_cleaned.shape}")
print(tmdb_cleaned.head(3))


✅ Filtered TMDb movies: (2666, 20)
✅ Cleaned TMDb dataset shape: (2666, 33)
                     title     budget     revenue  popularity  vote_average  \
0  Avengers: Age of Ultron  365000000  1405403694     51.3514         7.270   
1                  Spectre  245000000   880707597     48.1287         6.564   
3           American Ultra   28000000    27100000     44.3710         6.089   

   vote_count  director_encoded  top_cast_avg_rating  release_year  \
0       23812               711             4.707700          2015   
1       10889              1218             3.377500          2015   
3        2866              1013             3.550833          2015   

   release_month  ...  History  Horror  Music  Mystery  Romance  \
0              4  ...        0       0      0        0        0   
1             10  ...        0       0      0        0        0   
3              8  ...        0       0      0        0        0   

   Science Fiction  TV Movie  Thriller  War  Western  
0 

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 🎯 Step 1: Define the Kaggle dataset column structure
kaggle_columns = [
    'title', 'budget', 'revenue', 'popularity', 'vote_average', 'vote_count',
    'director_encoded', 'top_cast_avg_rating', 'release_year', 'release_month',
    'budget_log', 'revenue_log', 'profit_ratio',
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
    'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
    'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
]

# 🎯 Step 2: Filter TMDB data (only valid rows)
tmdb_filtered = tmdb_df[(tmdb_df['budget'] > 0) & (tmdb_df['revenue'] > 0)].copy()
print(f"✅ Filtered TMDb movies: {tmdb_filtered.shape}")

# 🎯 Step 3: Clean & convert columns
tmdb_filtered['release_year'] = tmdb_filtered['release_year'].astype(int)
tmdb_filtered['release_month'] = tmdb_filtered['release_month'].astype(int)

# Encode directors (convert names → numbers)
if tmdb_filtered['director_encoded'].dtype == 'object':
    le = LabelEncoder()
    tmdb_filtered['director_encoded'] = tmdb_filtered['director_encoded'].fillna('Unknown')
    tmdb_filtered['director_encoded'] = le.fit_transform(tmdb_filtered['director_encoded'])
else:
    tmdb_filtered['director_encoded'] = tmdb_filtered['director_encoded'].fillna(0).astype(int)

# Fill missing cast ratings
tmdb_filtered['top_cast_avg_rating'] = tmdb_filtered['top_cast_avg_rating'].fillna(
    tmdb_filtered['top_cast_avg_rating'].mean()
)

# Compute logs & ratios
tmdb_filtered['budget_log'] = np.log1p(tmdb_filtered['budget'])
tmdb_filtered['revenue_log'] = np.log1p(tmdb_filtered['revenue'])
tmdb_filtered['profit_ratio'] = (tmdb_filtered['revenue'] - tmdb_filtered['budget']) / tmdb_filtered['budget']
tmdb_filtered['profit_ratio'] = tmdb_filtered['profit_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# 🎯 Step 4: Create genre columns
genre_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
              'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
              'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

for g in genre_list:
    tmdb_filtered[g] = tmdb_filtered['genre_ids'].apply(lambda x: 1 if g.lower() in str(x).lower() else 0)

# 🎯 Step 5: Align column order to Kaggle data
tmdb_filtered = tmdb_filtered[kaggle_columns]

print("✅ TMDb dataset aligned successfully:", tmdb_filtered.shape)

# 🎯 Step 6: Merge with Kaggle dataset
combined_df = pd.concat([kaggle_df, tmdb_filtered], ignore_index=True)
print("✅ Final Combined Dataset Shape:", combined_df.shape)
combined_df.head(3)


✅ Filtered TMDb movies: (2666, 20)
✅ TMDb dataset aligned successfully: (2666, 33)
✅ Final Combined Dataset Shape: (52191, 34)


,title,budget,revenue,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,release_month,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,profit_ratio_log
0,Pirates of the Caribbean: At World's End,300000000.0,961000000.0,139.082615,6.9,4500.0,714,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,1.164192
1,John Carter,260000000.0,284139100.0,43.926995,6.1,2124.0,89,6.108333,2012.0,3.0,...,0,0,0,0,1,0,0,0,0,0.088782
2,Spider-Man 3,258000000.0,890871626.0,115.699814,5.9,3576.0,1838,6.271936,2007.0,5.0,...,0,0,0,0,0,0,0,0,0,1.239241


In [33]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52191 entries, 0 to 52190
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                52191 non-null  object 
 1   budget               52191 non-null  float64
 2   revenue              52191 non-null  float64
 3   popularity           52191 non-null  float64
 4   vote_average         52191 non-null  float64
 5   vote_count           52191 non-null  float64
 6   director_encoded     52191 non-null  int64  
 7   top_cast_avg_rating  52191 non-null  float64
 8   release_year         52191 non-null  float64
 9   release_month        52191 non-null  float64
 10  budget_log           52191 non-null  float64
 11  revenue_log          52191 non-null  float64
 12  profit_ratio         52191 non-null  float64
 13  Action               52191 non-null  int64  
 14  Adventure            52191 non-null  int64  
 15  Animation            52191 non-null 

In [34]:
# Compute log for any missing rows
combined_df['profit_ratio_log'] = combined_df['profit_ratio_log'].fillna(
    np.log1p(combined_df['profit_ratio'])
)

# Verify no missing
print("✅ Missing profit_ratio_log:", combined_df['profit_ratio_log'].isna().sum())
combined_df.info()


✅ Missing profit_ratio_log: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52191 entries, 0 to 52190
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                52191 non-null  object 
 1   budget               52191 non-null  float64
 2   revenue              52191 non-null  float64
 3   popularity           52191 non-null  float64
 4   vote_average         52191 non-null  float64
 5   vote_count           52191 non-null  float64
 6   director_encoded     52191 non-null  int64  
 7   top_cast_avg_rating  52191 non-null  float64
 8   release_year         52191 non-null  float64
 9   release_month        52191 non-null  float64
 10  budget_log           52191 non-null  float64
 11  revenue_log          52191 non-null  float64
 12  profit_ratio         52191 non-null  float64
 13  Action               52191 non-null  int64  
 14  Adventure            52191 non-null  int64  
 15  Animat

In [35]:
combined_df.to_csv("combined_all_data.csv")